In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import normal
from keras.utils import plot_model
from deap import base, creator, tools, algorithms
from deap.benchmarks.tools import diversity, convergence, hypervolume
from functools import partial
import numpy as np
import random

Using TensorFlow backend.


In [2]:
def build_model(n_layers, input_dim, neurons, activation='sigmoid', initializer=None):
    if isinstance(neurons, list):
        assert len(neurons) == n_layers
    else:
        neurons = [neurons] * n_layers
        
    if initializer is None:
        # Uses normal initializer
        initializer = normal(mean=0, stddev=0.1, seed=13)
        
    model = Sequential()
    
    # Adds first hidden layer with input_dim parameter
    model.add(Dense(units=neurons[0], 
                    input_dim=input_dim, 
                    activation=activation,
                    kernel_initializer=initializer, 
                    name='hidden_layer'))
    
    
    # Adds output layer
    model.add(Dense(units=2, activation=activation, kernel_initializer=initializer, name='net_output'))
    
    # Compiles the model
    model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mean_squared_error'])
    
    return model

In [5]:
data_dim = 7
x_train = np.random.random((1000, data_dim))

In [6]:
x_train.shape

(1000, 7)

In [7]:
# randomly sellect one array
x_train[np.random.randint(x_train.shape[0], size=1), :]

array([[0.5932604 , 0.366529  , 0.05670106, 0.75522734, 0.09669258,
        0.57365789, 0.83205914]])

In [3]:
model = build_model(n_layers=1, input_dim=7, neurons=5)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
ind = np.concatenate(tuple(weight.flatten() for weight in model.get_weights())).tolist()

In [22]:
ind[50:]

[0.0, 0.0]

In [6]:
tuple(weight.flatten() for weight in model.get_weights())

(array([ 0.01745166, -0.0225782 ,  0.01967431,  0.0387477 ,  0.0251386 ,
         0.00515667, -0.01068268, -0.12614495, -0.02336994, -0.10256433,
        -0.05181826, -0.03860423, -0.13391905, -0.06569492,  0.14408804,
         0.08051246,  0.07736253,  0.01508499,  0.02778573, -0.08993146,
         0.13227466,  0.0797508 ,  0.0829831 ,  0.01504011, -0.09476091,
        -0.07435729, -0.00698865, -0.06255804, -0.1166985 , -0.01686729,
         0.06776601, -0.03450424,  0.17205839, -0.02406461,  0.0395343 ],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32),
 array([ 0.01745166, -0.0225782 ,  0.01967431,  0.0387477 ,  0.0251386 ,
         0.00515667, -0.01068268, -0.12614495, -0.02336994, -0.10256433],
       dtype=float32),
 array([0., 0.], dtype=float32))

In [9]:
type(model).__name__

'Sequential'

In [ ]:
# Plot model graph
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
model.predict(x_train)

In [ ]:
model.predict(x_train[np.random.randint(x_train.shape[0], size=1), :])

In [ ]:
model.predict(x_train[np.random.randint(x_train.shape[0], size=1), :])[0]

In [ ]:
print(model.summary())

In [10]:
model.get_weights()

[array([[ 0.01745166, -0.0225782 ,  0.01967431,  0.0387477 ,  0.0251386 ],
        [ 0.00515667, -0.01068268, -0.12614495, -0.02336994, -0.10256433],
        [-0.05181826, -0.03860423, -0.13391905, -0.06569492,  0.14408804],
        [ 0.08051246,  0.07736253,  0.01508499,  0.02778573, -0.08993146],
        [ 0.13227466,  0.0797508 ,  0.0829831 ,  0.01504011, -0.09476091],
        [-0.07435729, -0.00698865, -0.06255804, -0.1166985 , -0.01686729],
        [ 0.06776601, -0.03450424,  0.17205839, -0.02406461,  0.0395343 ]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32),
 array([[ 0.01745166, -0.0225782 ],
        [ 0.01967431,  0.0387477 ],
        [ 0.0251386 ,  0.00515667],
        [-0.01068268, -0.12614495],
        [-0.02336994, -0.10256433]], dtype=float32),
 array([0., 0.], dtype=float32)]

In [ ]:
model.get_weights()[0].shape

In [ ]:
model.get_weights()[2].shape

In [ ]:
weights = np.concatenate(
                (
                    model.get_weights()[0].flatten(),
                    model.get_weights()[2].flatten()
                )
            )

In [ ]:
weights[:35].reshape((7, 5))

In [ ]:
weights[-10:].reshape((5, 2))

In [ ]:
weights

In [ ]:
model.get_weights()[2].flatten()

In [ ]:
model.get_layer(name='hidden_layer')

In [ ]:
# including the bias weights
for l in model.layers:
    print(l.get_config())
    print(l.get_weights())

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMax, model=None)

In [ ]:
def initIndividual(cls, model):
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in model.get_weights()]
    return cls(weights)

In [ ]:
toolbox = base.Toolbox()
toolbox.register("individual", initIndividual, creator.Individual, model=model)

In [ ]:
ind1 = toolbox.individual()

In [ ]:
ind1.fitness.values = (1.0, 1.1)
print(ind1.fitness)

In [ ]:
mutant = toolbox.clone(ind1)
ind2, = tools.mutGaussian(mutant, mu=0.0, sigma=0.2, indpb=0.2)
del mutant.fitness.values

In [ ]:
def mutateIndividual(individual):
    mutant = toolbox.clone(ind1)
    mutated_ind = [tools.mutGaussian(weight, mu=0.0, sigma=0.2, indpb=0.9) for weight in mutant]
    del mutant.fitness.values
    return mutated_ind

In [ ]:
ind1

In [ ]:
mutateIndividual(ind1)

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
pop = toolbox.population(n=20)

In [ ]:
pop[0]

In [ ]:
pop[1]

In [ ]:
def eval_function(model, chromosome):
    x_data = np.random.random((1, 8))
    net_output = model.predict(x_data)[0]
    return (net_output[0]*np.random.random_sample(), net_output[1]*np.random.random_sample(),)

In [ ]:
def mutate_individual(individual):
    for i, weight in enumerate(individual):
        w, = tools.mutFlipBit(weight.flatten(), indpb=0.3)
        individual[i] = w.reshape(weight.shape)
    return individual

In [ ]:
def mate_individuals(ind1, ind2):
    for i, (w1, w2) in enumerate(zip(ind1, ind2)):
        cx1, cx2 = tools.cxTwoPoint(w1.flatten(), w2.flatten())
        ind1[i], ind2[i] = cx1.reshape(w1.shape), cx2.reshape(w2.shape)
    return ind1, ind2

In [ ]:
"""Multiobjective optmization"""
model = build_model(1, 8, 5)
# Creating the appropriate type of the problem
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list,
               fitness=creator.FitnessMax, model=None)

def initIndividual(cls, model):
    weights = [np.random.permutation(w.flat).reshape(
        w.shape) for w in model.get_weights()]
    return cls(weights)

toolbox = base.Toolbox()
history = tools.History()
toolbox.register("individual", initIndividual,
                 creator.Individual, model=model)

 # register the crossover operator
toolbox.register('mate', mate_individuals)
# register the mutation operator
toolbox.register('mutate', mutate_individual)
# register the evaluation function
toolbox.register('evaluate', partial(eval_function, model))
# register NSGA-II multiobjective optimization algorithm
toolbox.register("select", tools.selNSGA2)
 # instantiate the population
toolbox.register('population', tools.initRepeat,
                 list, toolbox.individual)

# maintain stats of the evolution
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('avg', np.mean)
stats.register('std', np.std)
stats.register('min', np.min)
stats.register('max', np.max)

logbook = tools.Logbook()
logbook.header = "gen", "evals", "std", "min", "avg", "max"

# create an initial population of N individuals
pop = toolbox.population(n=20)
history.update(pop)

def eq(ind1, ind2):
    return np.array_equal(ind1[0], ind2[0])
    

# object that contain the best individuals
hof = tools.ParetoFront(eq)

# Evaluate the individuals with an invalid fitness
invalid_ind = [ind for ind in pop if not ind.fitness.valid]

In [ ]:
fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)

In [ ]:
for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

In [ ]:
# This is just to assign the crowding distance to the individuals
# no actual selection is done
pop = toolbox.select(pop, len(pop))

record = stats.compile(pop)
logbook.record(gen=0, evals=len(invalid_ind), **record)
print(logbook.stream)
hof.update(pop)

In [ ]:
best_inds, best_inds_fitness = [], []

# Begin the generational process
for gen in range(1, 21):
    # Vary the population
    offspring = tools.selTournamentDCD(pop, len(pop))
    offspring = [toolbox.clone(ind) for ind in offspring]

    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if random.random() <= 0.9:
            toolbox.mate(ind1, ind2)

        toolbox.mutate(ind1)
        toolbox.mutate(ind2)
        del ind1.fitness.values, ind2.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    best_ind = tools.selBest(pop, 1)[0]
    best_inds.append(best_ind) # add the best individual for each generation
    best_inds_fitness.append(best_ind.fitness.values)

    # Select the next generation population
    pop = toolbox.select(pop + offspring, len(offspring))
    record = stats.compile(pop)
    logbook.record(gen=gen, evals=len(invalid_ind), **record)
    hof.update(pop)
    
print("Final population hypervolume is %f" % hypervolume(pop, [11.0, 11.0]))

In [ ]:
print(logbook.stream)

In [ ]:
offspring = tools.selTournamentDCD(pop, len(pop))
offspring = [toolbox.clone(ind) for ind in offspring]

In [ ]:
ind1 = offspring[0]
ind2 = offspring[1]

In [ ]:
def mate_individuals(ind1, ind2):
    for i, w1, w2 in enumerate(zip(ind1, ind2)):
        cx1, cx2 = tools.cxTwoPoint(w1.flatten(), w2.flatten())
        ind1[i], ind2[i] = cx1.reshape(w1.shape), cx2.reshape(w2.shape)
    return ind1, ind2

In [ ]:
ind1, ind2 = mate_individuals(ind1, ind2)

In [ ]:
type(ind2)

In [ ]:
ind1 = mutate_individual(ind1, 0.2)

In [ ]:
ind1.history_index

In [ ]:
toolbox.mate(ind1, ind2)

In [ ]:
ind1.__dict__

In [ ]:
d = np.array([[0.25773812, 0.99876807, 0.98062046, 0.16666008, 0.19844747,
        0.85773822, 0.13071776],]).reshape((7,))

In [ ]:
model.predict(d)

In [ ]:
d.shape

In [ ]:
data = np.array([0.25773812, 0.99876807, 0.98062046, 0.16666008, 0.19844747, 0.85773822, 0.13071776])
data = data.reshape((1, 7))
print(data.shape)
print(data)

In [ ]:
model.predict(data)

In [9]:
def init_individual(cls, model):
    ind = cls(np.concatenate(
        (
            model.get_weights()[0].flatten(),
            model.get_weights()[2].flatten()
        )
    ))

    ind.shape_1 = model.get_weights()[0].shape
    ind.shape_2 = model.get_weights()[2].shape

    return ind

# Creating the appropriate type of the problem
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", np.ndarray,
               fitness=creator.FitnessMax, model=None)

toolbox = base.Toolbox()
history = tools.History()

toolbox.register("individual", init_individual,
                 creator.Individual, model=model)

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [10]:
ind1 = toolbox.individual()

In [11]:
ind1.__dict__

{'fitness': deap.creator.FitnessMax(()), 'shape_1': (7, 5), 'shape_2': (5, 2)}

In [12]:
ind1.__class__

deap.creator.Individual

In [15]:
ind1

Individual([ 0.01745166, -0.0225782 ,  0.01967431,  0.0387477 ,
             0.0251386 ,  0.00515667, -0.01068268, -0.12614495,
            -0.02336994, -0.10256433, -0.05181826, -0.03860423,
            -0.13391905, -0.06569492,  0.14408804,  0.08051246,
             0.07736253,  0.01508499,  0.02778573, -0.08993146,
             0.13227466,  0.0797508 ,  0.0829831 ,  0.01504011,
            -0.09476091, -0.07435729, -0.00698865, -0.06255804,
            -0.1166985 , -0.01686729,  0.06776601, -0.03450424,
             0.17205839, -0.02406461,  0.0395343 ,  0.01745166,
            -0.0225782 ,  0.01967431,  0.0387477 ,  0.0251386 ,
             0.00515667, -0.01068268, -0.12614495, -0.02336994,
            -0.10256433], dtype=float32)

In [23]:
class Ind(object): 
    pass

In [46]:
ind = Ind()

In [47]:
setattr(ind, 'weights', [tuple(weights.shape) for weights in model.get_weights()])

In [48]:
ind.weights

[(7, 5), (5,), (5, 2), (2,)]

In [49]:
genome = list(np.concatenate(tuple(weight.flatten() for weight in model.get_weights())).tolist())

In [50]:
genome

[0.017451664432883263,
 -0.022578200325369835,
 0.019674310460686684,
 0.038747698068618774,
 0.025138601660728455,
 0.005156669765710831,
 -0.010682681575417519,
 -0.12614494562149048,
 -0.023369936272501945,
 -0.10256432741880417,
 -0.05181825906038284,
 -0.03860422968864441,
 -0.13391904532909393,
 -0.06569492071866989,
 0.1440880447626114,
 0.08051245659589767,
 0.0773625299334526,
 0.015084986574947834,
 0.027785733342170715,
 -0.08993145823478699,
 0.13227465748786926,
 0.07975079864263535,
 0.08298309892416,
 0.015040106140077114,
 -0.09476090967655182,
 -0.0743572860956192,
 -0.006988647859543562,
 -0.06255804002285004,
 -0.1166984960436821,
 -0.01686728559434414,
 0.06776601076126099,
 -0.0345042422413826,
 0.17205838859081268,
 -0.02406460791826248,
 0.039534296840429306,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.017451664432883263,
 -0.022578200325369835,
 0.019674310460686684,
 0.038747698068618774,
 0.025138601660728455,
 0.005156669765710831,
 -0.010682681575417519,
 -0.1261449456

In [39]:
[(tuple(weights.shape), len(weights.flatten())) for weights in model.get_weights()]

[((7, 5), 35), ((5,), 5), ((5, 2), 10), ((2,), 2)]

In [62]:
weights = [
    np.array(genome[:35]).reshape(ind.weights[0]),
    np.array(genome[35:40]).reshape(ind.weights[1]),
    np.array(genome[40:50]).reshape(ind.weights[2]),
    np.array(genome[-2:]).reshape(ind.weights[3]),
]

In [57]:
genome[35:40]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [61]:
genome[-3:]

[-0.10256432741880417, 0.0, 0.0]

In [63]:
weights

[array([[ 0.01745166, -0.0225782 ,  0.01967431,  0.0387477 ,  0.0251386 ],
        [ 0.00515667, -0.01068268, -0.12614495, -0.02336994, -0.10256433],
        [-0.05181826, -0.03860423, -0.13391905, -0.06569492,  0.14408804],
        [ 0.08051246,  0.07736253,  0.01508499,  0.02778573, -0.08993146],
        [ 0.13227466,  0.0797508 ,  0.0829831 ,  0.01504011, -0.09476091],
        [-0.07435729, -0.00698865, -0.06255804, -0.1166985 , -0.01686729],
        [ 0.06776601, -0.03450424,  0.17205839, -0.02406461,  0.0395343 ]]),
 array([0., 0., 0., 0., 0.]),
 array([[ 0.01745166, -0.0225782 ],
        [ 0.01967431,  0.0387477 ],
        [ 0.0251386 ,  0.00515667],
        [-0.01068268, -0.12614495],
        [-0.02336994, -0.10256433]]),
 array([0., 0.])]